In [16]:
import numpy as np 
import pandas as pd
import cplex # community edition
import matplotlib.pyplot as plt
import copy
from sklearn.externals import joblib # 保存及加载变量

第一阶段

In [51]:
arg_source = pd.read_excel('data/step1-ANSI.xlsx')
arg_source.head()

# 参数和决策变量
price = arg_source['单箱价格'].values
sold = arg_source['上期销售量'].values 
cap = arg_source['产能'].values

sold_obj = 2400   # 目标销量(箱)
price_obj = 42000 # 目标结构(元/箱)
I = price.size

In [52]:
var = [i for i in map(lambda x: 'x(' + str(x) + ')', list(range(arg_source.shape[0])))] # ['x(0)', 'x(1)', ...]
c = price/(250*sold_obj*price_obj) # (Σc_i*x_i)^2中的c
lp = '' # lp文件内容

# p1: 一次项 x1 x2 x3 ...
p1 = ''
for i in range(len(cap)):
    p1 += ' - %s x(%s)'%((2*I*price/(250*sold_obj*price_obj) + 2/sold)[i], i)

# p2: 二次项 x1^2 x2^2 ... (已经乘2)
p2 = ''
for i in range(len(cap)):
    p2 += ' + %s x(%s)^2'%((2 * I * c**2 + 2/(sold**2))[i], i)

# p3: 交叉二次项 x1x2 x1x3 ... (已经乘2)
p3 = ''
for i in range(len(cap)):
    for j in range(i+1, len(cap)):
        p3 += ' + %s x(%s) * x(%s)'%(4*I*c[i]*c[j],i,j)

lp += ('Minimize\n obj:' + p1 + ' +[' + p2 + p3 + ']/2\n')

# 约束条件

# p4: Σx_i
p4 = ''
for i in range(len(cap)):
    p4 += ' + %s x(%s)'%(1/250, i)

lp += '\nSubject To\n'
lp += ' c1: %s >= %s \n'%(p4, 0.9*sold_obj)
lp += ' c2: %s <= %s \n'%(p4, 1.1*sold_obj)
for i in range(len(cap)):
    lp += ' c%s: x(%s) - Rgc%s = 0\n'%(3+i, i, i)
lp += 'Bounds\n'
for i in range(len(cap)): 
    lp += 'x(%s) Free\n'%i
for i in range(len(cap)): 
    lp += ' 0 <= Rgc%s <= %s \n'%(i, cap[i])
lp += 'General\n'
for i in range(len(cap)): lp += ' %s'%(var[i])
lp += '\nEnd'

with open('write.lp', 'w') as f:
    f.write(lp)

my_prob = cplex.Cplex('write.lp')
# my_prob = cplex.Cplex('lp/配置 2-2.lp')
my_prob.solve()

x = my_prob.solution.get_values()
sold_next = np.array(x, dtype='int')[0:int(len(x)/2)]
sold_next

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIQP Presolve eliminated 72 rows and 71 columns.
Reduced MIQP has 1 rows, 71 columns, and 71 nonzeros.
Reduced MIQP has 0 binaries, 71 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 5041 nonzeros.
Presolve time = 0.00 sec. (0.57 ticks)
Tried aggregator 1 time.
Reduced MIQP has 1 rows, 71 columns, and 71 nonzeros.
Reduced MIQP has 0 binaries, 71 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 5041 nonzeros.
Presolve time = 0.00 sec. (0.43 ticks)
Classifier predicts products in MIQP should be linearized.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.02 sec. (4.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best B

array([    15,    184,    154,    476,     81,   4798,    525,    597,
         8219,    718,    694,   2849,   2009,    238,   9065,   9569,
        13868,    384,   7020,     33,     35,     19,    152,   9789,
       107901,    759,  11719,  69270,  50299,  46716,   2902,  32094,
         7422,  51315,  15404,   9070,   7977,    457,    112,   1731,
         1996,   1898,  47055,    574,    621,   1359,     78,   3638,
         1292,  10218,     78,   4262,   2155,   1690,     40,    136,
           90,     67,   2857,   1769,   2680,    615,     44,    160,
          403,    357,    859,    336,   4322,  10355,   6971])

In [47]:
# 销量偏差百分比
delta_sold = (sold_obj - sold_next.sum()/250)/sold_obj
delta_price = (price_obj - (sold_next*price).sum()/(sold_next.sum()))/price_obj
delta_sale = (sold_obj*price_obj - (sold_next*price).sum()/250)/(sold_obj*price_obj)
delta_change = ((sold_next-sold)/sold).mean()
print('销量偏差：%.2f \n结构偏差：%.2f\n销售额偏差：%.2f\n销量变动程度：%.2f'%(delta_sold, delta_price, delta_sale, delta_change))

销量偏差：0.04 
结构偏差：0.02
销售额偏差：0.06
销量变动程度：0.05


In [77]:
arg_source['本期销售目标'] = sold_next
arg_source['本期预计销量'] = None
arg_source.head()

# 如果所有产能用满：销量2600，结构41678

,品规名,单箱价格,品规ID,上期销售量,产能,本期销售目标,本期预计销量,调整档位数
0,中南海(京韵细支烤烟),75000,6901028010290,14,42,15,None,1
1,云烟(中支金腰带),75000,6901028340670,183,243,184,None,8
2,云烟(细支大重九),250000,6901028312356,153,186,154,None,1
3,云烟(细支珍品),65000,6901028339537,474,580,476,None,8
4,云烟(软大重九),245000,6901028051989,80,122,81,None,1


## 第二阶段

In [21]:
slym = pd.read_excel('data/step2-ANSI-new.xlsx')
slym.head()

,Unnamed: 0,品规ID,档位,订足面,订足率,订单满足率,订购率,上期策略,总户数,订购量,需求量,投放量,订足户数,订购户数,品规名
0,0,6901028010290,二十九档,0.500000,0.00303,0.500000,0.009091,2,660,4,8,1320,3,6,中南海(京韵细支烤烟)
1,1,6901028010290,二十七档,0.142857,NaN,0.222222,0.020349,0,344,2,9,0,1,7,中南海(京韵细支烤烟)
2,2,6901028010290,二十档,0.000000,NaN,0.000000,0.002648,0,1133,0,3,0,0,3,中南海(京韵细支烤烟)
3,3,6901028010290,二十八档,0.000000,0.00000,0.000000,0.009202,1,326,0,3,326,0,3,中南海(京韵细支烤烟)
4,4,6901028010290,十四档,0.000000,NaN,0.000000,0.000934,0,1071,0,13,0,0,1,中南海(京韵细支烤烟)


In [22]:
def preprocess(dt):
    '''
    处理三率一面数据框中的缺失值
    将各品规的档位按30到1排列
    返回处理后的数据框
    '''

    # 字符串“xx档”转为int
    name_list = ['三十档', '二十九档', '二十八档', '二十七档', '二十六档','二十五档','二十四档','二十三档','二十二档','二十一档',
        '二十档','十九档','十八档','十七档','十六档','十五档','十四档', '十三档','十二档','十一档', '十档','九档',
        '八档','七档','六档', '五档','四档','三档','二档','一档']

    t = 30
    for i in name_list: 
        dt['档位'] = dt['档位'].replace(i,t)
        t -= 1
    
    # 处理inf和Na
    null_rate = dt.isnull().sum().sum()/(dt.shape[0]*dt.shape[1]) # 三率一面数据中数据缺失率
    dt = dt.replace(np.inf, 1) 
    dt['上期策略'].fillna(0, inplace=True) # '上期策略'的缺失用0填补,因为输出策略是据此调整的,用均值影响太大
    dt['上期策略'] = dt['上期策略'].astype(int) # '上期策略'需要转成int
    for column in list(dt.columns[dt.isnull().sum() > 0]):
        mean_val = dt[column].mean()
        dt[column].fillna(mean_val, inplace=True)

    # 将各品规的档位按30到1排列
    dt.sort_values(by=['品规ID','档位'],inplace=True,ascending=[True,False])  
    return dt  
slym = preprocess(slym)

In [23]:
def get_guide(slym, item, itemid):

    # 由step1的'品规ID'筛选出该品规的三率一面数据框
    item_cube = slym[slym['品规ID'] == itemid]

    strategy = item_cube['上期策略'].values
    dzm = item_cube['订足面'].values
    dzl = item_cube['订足率'].values
    ddmz = item_cube['订单满足率'].values
    dgl = item_cube['订购率'].values
    dg = item_cube['订购量'].values              
    number = item_cube['总户数'].values           
    number_z = item_cube['订足户数'].values       
    number_c = item_cube['订购户数'].values       

    return {'strategy':strategy, 
            'dzm':dzm, 'dzl':dzl,'ddmz':ddmz, 'dgl':dgl,
            'item':item, 'itemid':itemid, 
             'number':number, 'number_z':number_z, 'number_c':number_c}

In [100]:
alpha = 1.25 # alpha=1.25 销量-投放膨胀因子，增加1单位销量需要增加1*alpha投放量
beta  = 0.3  # beta=0.3 市场反馈评分最小调节单位，每单位策略对评分的影响大小

w = [3,3,1,0.1] #三率一面的默认权重 
o = [0.5,0.5,0.8,0.5] #三率一面的默认目标

def single_strategy(guide, obj): 
    '''
    单品规投放策略
    输入：单个品规的三率一面; 该品规的销量改变目标(条)
    输出：list [ID, name, 30', 29', ...]
    '''
    # 浅拷贝,否则相当于传址调用,会修改guide_data['strategy']
    strategy = copy.copy(guide['strategy']) 

    # 饱和度得分: 三率一面、档位优先级，顺序为30~1档
    score = np.array([w[0]*(o[0]-guide['dzm'][i])+w[1]*(guide['dzl'][i]-o[1])
            +w[2]*(o[2]-guide['ddmz'][i])+w[3]*(guide['dgl'][i]-o[3])-0.12*i for i in range(30)])

    score = (score - score.mean()) * np.arange(1, 0.4, -0.02)

    if obj > 0:
        delta = alpha * obj 
        for i in range(0,30): # 从30到1档
            while score[i] < -0.8: # 过于饱和的
                if strategy[i] <= 0:
                    break
                strategy[i] -= 1
                delta += guide['number_c'][i]
                score[i] += beta 
        while delta > 0:
            t = np.where(score==score.max())[0][0]
            strategy[t] += 1
            score[t] -= beta
            delta -= guide['number_c'][t] # 这里要用
    elif obj < 0:
        delta = obj/alpha
        for i in range(0,30):
            while score[i] > 0.8:
                strategy[i] += 1
                delta -= guide['number_c'][i]
                score[i] -= beta
        n = 1
        while delta < 0 and n <= 30:
            n += 1
            t = np.where(score==score.min())[0][0]
            if strategy[t] > 0:
                strategy[t] -= 1
                score[t] += beta
                delta += guide['number_c'][t]
            else:
                score[t] += 10000
    
    output = np.append(np.array([guide['item'],guide['itemid']]), strategy)
    return strategy, output

In [101]:
# 创建结果空数据框
col = ['品规ID','品规名','三十档', '二十九档', '二十八档', '二十七档', '二十六档','二十五档','二十四档','二十三档',
        '二十二档','二十一档','二十档','十九档','十八档','十七档','十六档','十五档','十四档', '十三档',
        '十二档','十一档', '十档','九档','八档','七档','六档', '五档','四档','三档','二档','一档']
strategy = pd.DataFrame(columns=col)

for i in range(arg_source.shape[0]):
    guide = get_guide(slym, arg_source.iloc[i,0], arg_source.iloc[i,2])

    single = single_strategy(guide, arg_source.iloc[i,5]-arg_source.iloc[i,3])
    strategy.loc[i,:] = single[1] # 生成策略

    single[0]-guide['strategy']
    
    pred_sale = (guide['dzl']*guide['number']*single[0]).sum()
    arg_source.iloc[i,6] = round(pred_sale) # 预估销量



# 评估--------------------------------------
# 上期销售，本期输入目标，本期分解目标，本期预计
pred_sale = arg_source['本期预计销量'].values.sum()/250
pred_price = (arg_source['本期预计销量'].values/250 * price).sum()/pred_sale

print('上期销量：%.1f; 上期结构：%.1f'%(sold.sum()/250,  (sold*price).sum()/sold.sum() ))
print('本期输入销量：%s; 本期输入结构：%s'%(sold_obj, price_obj) )
print('本期分解销量：%.1f; 本期分解结构：%.1f'%(sold_next.sum()/250, (sold_next*price).sum()/sold_next.sum() ))
print('本期预计销量：%.1f; 本期预计结构：%.1f'%(pred_sale, pred_price) )

上期销量：2163.1; 上期结构：41590.8
本期输入销量：2400; 本期输入结构：42000
本期分解销量：2382.5; 本期分解结构：41147.3
本期预计销量：2385.4; 本期预计结构：41582.0
